In [ ]:
import pymysql
from configparser import ConfigParser

config = ConfigParser()
config.read('../Chapter1/config.ini')

connection = pymysql.connect(
    host=config.get('DB', 'host'),
    user=config.get('DB', 'user'),
    password=config.get('DB', 'password'),
    port=config.getint('DB', 'port'),
    cursorclass=pymysql.cursors.DictCursor,
)

with connection.cursor() as cursor:
    sql = """
    CREATE DATABASE IF NOT EXISTS chapter3;
    """
    cursor.execute(sql)
    

In [ ]:
connection = pymysql.connect(
    host=config.get('DB', 'host'),
    user=config.get('DB', 'user'),
    password=config.get('DB', 'password'),
    port=config.getint('DB', 'port'),
    cursorclass=pymysql.cursors.DictCursor,
    db='chapter3',
)

# 建立有時間紀錄的表格
with connection.cursor() as cursor:
    sql = """
        CREATE TABLE IF NOT EXISTS users2 (
            id INT AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(255) NOT NULL,
            age INT NOT NULL,
            username VARCHAR(255) NOT NULL,
            password VARCHAR(255) NOT NULL,

            
        );
    """
    cursor.execute(sql)


In [ ]:
# 取得當前時間並插入資料


with connection.cursor() as cursor:
    sql = """
        INSERT INTO users2 (name, age, username, password)
        VALUES ("Jarvis", 25, "jarvis", "123456");
    """
    # 這裡使用標準 SQL 日期格式 'YYYY-MM-DD HH:MM:SS' 或是 datetime 物件都可以
    cursor.execute(sql, ())

    connection.commit()
